In [2]:
from token_tensor import token_embd

In [3]:
token_embd=token_embd[:6]

In [4]:
print(token_embd)

tensor([[-3.9299,  0.3435, -1.1167, -0.7704,  0.1609, -0.7244,  0.0543, -0.9665,
          0.6363, -0.0695, -0.8082, -0.0066, -0.3590,  0.3171, -0.8529, -1.4709],
        [ 0.8901,  1.3784,  0.3849,  2.3110,  0.6790, -0.3963, -0.8711,  0.2409,
          0.5548, -1.5825, -0.8137,  2.0478,  1.8619, -1.4766, -1.4558, -0.5568],
        [-0.0675, -0.3400,  0.3323, -0.3894, -0.4239, -1.5554, -0.7460, -1.0704,
          2.2066,  1.4488, -1.3420, -1.0142,  1.3992, -1.7187,  1.3786,  0.9301],
        [-0.1008, -0.4432,  1.6195,  0.9941, -0.6017,  0.0592, -0.1425,  1.3669,
         -0.7418, -0.0278, -0.1867,  1.4558, -0.6580,  0.1098, -0.3025,  0.3136],
        [ 0.6641, -0.7266, -1.4639,  0.5260,  0.5943, -0.2211, -0.0428, -0.1622,
          0.5565, -2.3402,  0.3933, -2.3626, -1.0919, -1.1649, -0.0882,  0.1234],
        [ 0.0621,  0.2654, -0.2779,  0.3462, -0.3347,  0.8016,  0.6414, -1.8797,
          0.4572,  0.5687,  0.0301,  2.6750,  2.3108,  0.1477, -0.1007,  0.7459]])


In [5]:
import torch
g=torch.Generator()
g.manual_seed(42)

In [6]:
Wq=torch.randn((16,16))
Wk=torch.randn((16,16))
Wv=torch.randn((16,16))

In [7]:
Q= token_embd @ Wq
K= token_embd @ Wk
V= token_embd @ Wv

In [11]:
# let t=3
q_3=Q[3].unsqueeze(0)

In [12]:
print(q_3)

tensor([[-0.7004, -2.1707,  0.4151, -7.1856,  0.2614,  1.9833,  3.1952, -1.3695,
          0.5131, -1.8403, -0.4321,  1.7967,  1.5983,  0.1381,  2.6512, -3.1915]])


In [20]:
scores=[]
for j in range(K.size(0)):
    scores.append((q_3 @ K[j]).item())
    

In [21]:
print(scores)

[-101.55579376220703, -9.867706298828125, -43.30801010131836, 6.19899320602417, 29.937894821166992, 18.58074951171875]


In [22]:
scores=torch.tensor(scores)

In [23]:
print(scores)

tensor([-101.5558,   -9.8677,  -43.3080,    6.1990,   29.9379,   18.5807])


In [25]:
a_t=torch.softmax(scores,dim=-1)

In [29]:
print(a_t)

tensor([0.0000e+00, 5.1599e-18, 1.5478e-32, 4.9014e-11, 9.9999e-01, 1.1686e-05])


In [30]:
import math
scores=scores/math.sqrt(16)
a_t=torch.softmax(scores,dim=-1)
print(a_t)

tensor([4.9828e-15, 4.4914e-05, 1.0511e-08, 2.4934e-03, 9.4236e-01, 5.5098e-02])


In [31]:
print(a_t.sum())

tensor(1.)


In [32]:
print(a_t.shape)

torch.Size([6])


In [33]:
a_t.unsqueeze(0)

tensor([[4.9828e-15, 4.4914e-05, 1.0511e-08, 2.4934e-03, 9.4236e-01, 5.5098e-02]])

In [34]:
outs= a_t @ V

In [35]:
print(outs)

tensor([-1.6238,  2.5772, -4.1723, -8.0000,  4.8044,  6.9814,  4.6777, -4.9531,
        -4.5638, -3.3295, -2.2899,  0.2094,  0.9730,  2.1824,  1.8295, -2.5532])


In [36]:
#computing outputs for all the tokens in parallel
S=(Q @ K.transpose(0,1))/math.sqrt(16)
A=torch.softmax(S,dim=-1)
outputs= A @ V
print(outputs)

tensor([[-3.8322, -3.4199, -5.0385,  3.8636,  4.7724,  2.4232, -0.7237,  5.9894,
         -0.2898, -5.8360,  7.3680, -2.4015, -7.9683,  5.3637,  4.9591,  1.6915],
        [-4.1738,  2.0860, -2.5238, 16.2751, -3.3240,  2.0053, -9.6921,  2.7263,
          5.3351, -3.3901,  4.8836,  1.3080, -7.1965,  2.1655,  0.2111,  2.1837],
        [-4.1843,  2.0892, -2.5107, 16.2264, -3.2698,  1.9808, -9.6388,  2.7093,
          5.2987, -3.3793,  4.8552,  1.2971, -7.1583,  2.2231,  0.2304,  2.1488],
        [-1.6238,  2.5772, -4.1723, -8.0000,  4.8044,  6.9814,  4.6777, -4.9531,
         -4.5638, -3.3295, -2.2899,  0.2094,  0.9730,  2.1824,  1.8295, -2.5532],
        [ 3.3951,  3.8272,  6.2025, -1.3070, -8.4392, -0.3390,  1.0025,  0.8619,
          3.4572,  4.5945, -2.3707, -2.4025,  5.5453, -1.7792,  2.9522, -6.0952],
        [-4.1739,  2.0860, -2.5236, 16.2742, -3.3231,  2.0049, -9.6912,  2.7260,
          5.3345, -3.3899,  4.8831,  1.3079, -7.1958,  2.1665,  0.2114,  2.1831]])


In [39]:
d_model=token_embd.size(1)
#let no of heads be 4
h=4
d_head=d_model//4
T=token_embd.size(0)

In [40]:
print(Q.shape)

torch.Size([6, 16])


In [41]:
Q=Q.view(T,h,d_head).transpose(0,1)
K=K.view(T,h,d_head).transpose(0,1)
V=V.view(T,h,d_head).transpose(0,1)

In [42]:
print(Q.shape)

torch.Size([4, 6, 4])


In [43]:
S_mh=(Q @ K.transpose(1,2))/math.sqrt(d_head)
A_mh=torch.softmax(S_mh,dim=-1)

In [44]:
print(A_mh.shape)

torch.Size([4, 6, 6])


In [45]:
outputs_mh= A_mh @ V

In [46]:
print(outputs_mh.shape)

torch.Size([4, 6, 4])


In [48]:
print(outputs_mh)

tensor([[[ -4.1736,   2.0846,  -2.5246,  16.2727],
         [ -4.1738,   2.0860,  -2.5238,  16.2751],
         [ -6.5517,   2.8063,   0.4275,   5.3405],
         [ -4.1737,   2.0859,  -2.5238,  16.2750],
         [  3.3955,   3.8274,   6.2028,  -1.3072],
         [ -2.2822,  -5.1332,  -6.1663,   2.9260]],

        [[  8.8679,  -3.5034,   2.3003,  -1.0826],
         [  8.3036,  -3.2483,   1.7453,  -0.9061],
         [ -3.3240,   2.0053,  -9.6921,   2.7262],
         [ -3.2531,   1.3998,  -5.8921,   2.8945],
         [  8.8678,  -3.5033,   2.3003,  -1.0826],
         [ -3.3240,   2.0053,  -9.6921,   2.7263]],

        [[  3.5856,  -0.4533,   2.7268,  -3.9519],
         [  5.3351,  -3.3901,   4.8836,   1.3080],
         [  5.3351,  -3.3901,   4.8836,   1.3080],
         [ -0.6399,   0.8256,  -2.4557,  -1.2220],
         [  3.5660,  -0.1238,   2.3812,  -3.7688],
         [ -4.5719,  -2.8441,  -2.3023,  -0.1625]],

        [[  5.5450,  -1.7795,   2.9521,  -6.0950],
         [  1.3814,  15.1

In [55]:
outputs_mh_cat=outputs_mh.transpose(0,1).contiguous().view(T,d_model)

In [56]:
print(outputs_mh_cat)

tensor([[ -4.1736,   2.0846,  -2.5246,  16.2727,   8.8679,  -3.5034,   2.3003,
          -1.0826,   3.5856,  -0.4533,   2.7268,  -3.9519,   5.5450,  -1.7795,
           2.9521,  -6.0950],
        [ -4.1738,   2.0860,  -2.5238,  16.2751,   8.3036,  -3.2483,   1.7453,
          -0.9061,   5.3351,  -3.3901,   4.8836,   1.3080,   1.3814,  15.1081,
           4.5548,  -5.6763],
        [ -6.5517,   2.8063,   0.4275,   5.3405,  -3.3240,   2.0053,  -9.6921,
           2.7262,   5.3351,  -3.3901,   4.8836,   1.3080,   1.3800,  15.1056,
           4.5538,  -5.6752],
        [ -4.1737,   2.0859,  -2.5238,  16.2750,  -3.2531,   1.3998,  -5.8921,
           2.8945,  -0.6399,   0.8256,  -2.4557,  -1.2220,   1.4705,   2.1971,
           1.9325,  -2.8404],
        [  3.3955,   3.8274,   6.2028,  -1.3072,   8.8678,  -3.5033,   2.3003,
          -1.0826,   3.5660,  -0.1238,   2.3812,  -3.7688, -11.0864,   1.3989,
           5.0457,   4.2451],
        [ -2.2822,  -5.1332,  -6.1663,   2.9260,  -3.3240,  

In [57]:
#masking out the future tokens are recomputing the attention
print(S_mh)

tensor([[[-16.3660,  18.3627,  15.9109,   0.8663,  -0.1749,  27.0407],
         [  7.2746,  37.1957,   3.7143, -25.7223,   2.2130,  50.8927],
         [-19.0089,   7.0316,  25.8919,   8.9600,  11.7991,   0.3773],
         [-35.8694,  -3.4605,  -8.9095,   8.9895,  -2.2200,  21.5980],
         [ 18.2924,  -2.2805,  -1.1965,  -1.2358,  -7.0884, -10.9573],
         [ 41.3735,  43.8532,  18.5848, -30.1637,  -2.5144,  33.2393]],

        [[ 17.4880,  10.6442,  36.2854,  -0.9803,  -5.8783,  -8.6813],
         [-16.3024,  16.4582,  27.0488, -16.0808,  10.0470,  24.0230],
         [-14.6557,   0.1517, -11.8092,  -6.2032,   1.8287,  13.9672],
         [ -0.7114,  -0.9930,  -8.9801,  -0.7400,  -5.0274,   0.6793],
         [  0.7032,  -2.5222,  14.2639,   1.7967,   2.2852,   0.4729],
         [-17.7540,  -2.2875, -12.4894,  -6.7099,   0.1854,  17.4037]],

        [[ -4.1437,  -3.9214,   2.7575,  19.2587, -13.2662, -10.3023],
         [-22.3140,   7.4903, -27.3430, -14.0370,   7.1135,  59.9320],
  

In [58]:
mask=torch.tril(torch.ones(T,T))
for i in range(h):
    S_mh[i]=S_mh[i].masked_fill(mask==0,float('-inf'))
print(S_mh)
    

tensor([[[-16.3660,     -inf,     -inf,     -inf,     -inf,     -inf],
         [  7.2746,  37.1957,     -inf,     -inf,     -inf,     -inf],
         [-19.0089,   7.0316,  25.8919,     -inf,     -inf,     -inf],
         [-35.8694,  -3.4605,  -8.9095,   8.9895,     -inf,     -inf],
         [ 18.2924,  -2.2805,  -1.1965,  -1.2358,  -7.0884,     -inf],
         [ 41.3735,  43.8532,  18.5848, -30.1637,  -2.5144,  33.2393]],

        [[ 17.4880,     -inf,     -inf,     -inf,     -inf,     -inf],
         [-16.3024,  16.4582,     -inf,     -inf,     -inf,     -inf],
         [-14.6557,   0.1517, -11.8092,     -inf,     -inf,     -inf],
         [ -0.7114,  -0.9930,  -8.9801,  -0.7400,     -inf,     -inf],
         [  0.7032,  -2.5222,  14.2639,   1.7967,   2.2852,     -inf],
         [-17.7540,  -2.2875, -12.4894,  -6.7099,   0.1854,  17.4037]],

        [[ -4.1437,     -inf,     -inf,     -inf,     -inf,     -inf],
         [-22.3140,   7.4903,     -inf,     -inf,     -inf,     -inf],
  

In [59]:
A_mh=torch.softmax(S_mh,dim=-1)
print(A_mh)

tensor([[[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [1.0126e-13, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [3.1613e-20, 6.4433e-09, 1.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [3.2963e-20, 3.9175e-06, 1.6847e-08, 1.0000e+00, 0.0000e+00,
          0.0000e+00],
         [1.0000e+00, 1.1622e-09, 3.4363e-09, 3.3038e-09, 9.4900e-12,
          0.0000e+00],
         [7.7289e-02, 9.2269e-01, 9.7975e-12, 6.6059e-33, 6.7275e-21,
          2.2671e-05]],

        [[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [5.9192e-15, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [3.7088e-07, 9.9999e-01, 6.3895e-06, 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [3.6676e-01, 2.7674e-01, 9.4045e-05, 3.5641e-01, 0.0000e+00,
          0.0000e+00],
         [1.2903e-06, 5.1272e-08, 9.9999e-01, 3.8509e-06, 6.2768e-06

In [60]:
outputs_mh= A_mh @ V
outputs_mh_cat=outputs_mh.transpose(0,1).contiguous().view(T,d_model)
print(outputs_mh_cat)

tensor([[  3.3955,   3.8274,   6.2028,  -1.3072,  -8.4398,  -0.3390,   1.0024,
           0.8619,   3.4574,   4.5948,  -2.3709,  -2.4025,   5.5455,  -1.7797,
           2.9521,  -6.0953],
        [ -2.7578,  -5.8839,  -7.2024,   3.2803,   3.1547,   4.7679,  -1.9201,
           8.7876,   0.7276,  -7.7620,  10.8712,  -2.8995, -11.6689,   1.5101,
           5.1194,   4.6072],
        [ -6.5517,   2.8063,   0.4275,   5.3405,   3.1547,   4.7678,  -1.9200,
           8.7876,   0.5687,  -7.4613,  10.3241,  -2.8218,   1.3814,  15.1088,
           4.5549,  -5.6765],
        [  2.2539,  -2.7370,  -1.1063,  -0.9482,  -3.1904,   0.4788,  -0.3487,
           3.1831,   2.8413,   4.0255,  -2.2499,  -2.2825,  -6.3280,  -2.9567,
          -1.3604,   1.2322],
        [  3.3955,   3.8274,   6.2028,  -1.3072,   8.8678,  -3.5033,   2.3003,
          -1.0826,   3.5487,  -0.0920,   2.3568,  -3.8183, -11.0867,   1.3989,
           5.0461,   4.2452],
        [ -2.2822,  -5.1332,  -6.1663,   2.9260,  -3.3240,  